## Preperatory calculations and processing

- Calculate the area grid and save
- Process all the data and calculate the hole areas per timestep and the max hole area per year, save as json

# Get the data

Need access to project on gadi or local copy, edit datadir below if not on gadi

In [ ]:
import glob, os
#Assume running on gadi or with /g/data mounted, eg with sshfs
datadir = '/g/data/p73/archive/non-CMIP/CMORised/CCMI2022/CSIRO-ARCCSS/ACCESS-CM2-Chem/refD2/r1i1p1f1/Aday/toz/gn/v20220822/'
files = sorted(glob.glob(datadir + "*.nc"))

In [ ]:
for f in files:
    print(os.path.basename(f))

In [ ]:
import numpy as np
import datetime
import math
import pathlib
import xarray as xr
import matplotlib.pyplot as plt

In [ ]:
#ds = xr.open_dataset(datadir + fn)
ds = xr.open_mfdataset(files, combine='nested', concat_dim="time")

In [ ]:
ds

### Calculate cell areas of the lat/lon grid

In [ ]:
grid_nc = None
if os.path.exists('earth_m2.nc'):
    grid_nc = xr.open_dataset('earth_m2.nc')

In [ ]:
"""
This will create a global grid of the approximate size of each grid square.
"""
def gridsize(lat1,lon_inc):
    #https://en.wikipedia.org/wiki/Haversine_formula
    #https://stackoverflow.com/questions/639695/how-to-convert-latitude-or-longitude-to-meters/11172685#11172685
    lon1=200
    import math
    lat2=lat1
    lon2=lon1+lon_inc

    R = 6378.137 # // Radius of earth in km
    dLat = lat2 * np.pi / 180 - lat1 * np.pi / 180
    dLon = lon2 * np.pi / 180 - lon1 * np.pi / 180
    a = np.sin(dLat/2) * np.sin(dLat/2) + np.cos(lat1 * np.pi / 180) * np.cos(lat2 * np.pi / 180) * np.sin(dLon/2) * np.sin(dLon/2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    d = R * c
    return d * 1000 #; // meters

if grid_nc is None:
    #boxlo,boxla=np.array(np.meshgrid(np.arange(-179.5,179.5,1),np.arange(-89.5,89.5,1)))
    #boxlo,boxla=np.array(mgrid)

    #Modified to use lat/lon grid from our data - first get min,max,inc
    lats = (float(ds['lat'].min()), float(ds['lat'].max()), float(ds['lat'][1]- ds['lat'][0]))
    print(lats, ds['lat'].shape)
    lons = (float(ds['lon'].min()), float(ds['lon'].max()), float(ds['lon'][1]- ds['lon'][0]))
    print(lons, ds['lon'].shape)
    #Need to add a tiny amount to end of range
    mgrid = np.meshgrid(np.arange(lons[0],lons[1]+0.0000001,lons[2]),
                        np.arange(lats[0],lats[1]+0.0000001,lats[2]))
    print(mgrid[0].shape)
    boxlo,boxla=np.array(mgrid)
    grid=gridsize(boxla,lons[2])
    print("A:",grid.shape)

    grid_nc = xr.DataArray(grid,coords={'lat':boxla[:,1],'lon':boxlo[1,:]},dims=['lat','lon'])
    #At the equator for longitude and for latitude anywhere, the following approximations are valid:
    #1deg ~= 111km = 111000m
    lat_size=110567 * lats[2] #in m - size of 1 degree
    grid_nc['m2'] = grid_nc * lat_size
    grid_nc = grid_nc['m2']
    grid_nc.to_netcdf('earth_m2.nc')

In [ ]:
#plt.pcolormesh(boxlo[1,:],boxla[:,1],grid_nc)
plt.pcolormesh(grid_nc['lon'],grid_nc['lat'],grid_nc['m2'])
plt.colorbar()
plt.show()

In [ ]:
grid_nc

In [ ]:
#Sanity check
#Earth's surface area: 510.1 million km²
earth_sa = 510.1
m2 = np.array(grid_nc['m2']).sum()
km2 = 1e-6 * m2
Mkm2 = 1e-6 * km2
error = abs(earth_sa - Mkm2)
print(f"{m2} m\n{km2} km²\n{Mkm2} million km²\nError {round(error / Mkm2 * 100, 2)}%")
#Check within ~1% tolerance
assert(error < (earth_sa * 0.01))

## Test threshold plot

In [ ]:
ts = 21481 #24/10/2018
#ts = 16348 #October 4, 2004
t = ds['time'][ts]
print(t)

In [ ]:
ttoz = np.array(ds['toz'][ts,:,:])

threshold = 220 * 1e-5 #220 DU threshold converted to M
print(threshold)
print(ttoz.min(), ttoz.max())
print(ttoz.shape)
below = ttoz < threshold
print(below.sum()) #sum() is count for boolean array
ttoz[ttoz < threshold] = 0.0
ttoz[ttoz >= threshold] = 1.0

fig = plt.figure(frameon=False)
fig.set_size_inches(4,3)

ax = plt.Axes(fig, [0., 0., 1., 1.])
#ax.set_axis_off()
fig.add_axes(ax)

ax.imshow(ttoz, aspect='auto');

In [ ]:
ttoz.shape

## Calibration plot

Ensure our lat/lon data is aligned

In [ ]:
latb = np.array(ds['lat_bnds'][ts,:,0])
lonb = np.array(ds['lon_bnds'][ts,:,0])

lgrid = np.array(np.meshgrid(lonb, latb))
lgrid.shape

#sumgrid = lgrid[0] + lgrid[1]
#sumgrid.shape
longrid = lgrid[0]
latgrid = lgrid[1]

In [ ]:
fig = plt.figure(frameon=False)
fig.set_size_inches(4,3)

ax = plt.Axes(fig, [0., 0., 1., 1.])
#ax.set_axis_off()
fig.add_axes(ax)

ax.imshow(latgrid, aspect='auto');

In [ ]:
fig = plt.figure(frameon=False)
fig.set_size_inches(4,3)

ax = plt.Axes(fig, [0., 0., 1., 1.])
#ax.set_axis_off()
fig.add_axes(ax)

ax.imshow(longrid, aspect='auto');

### Use the area grid to get the total area below threshold

In [ ]:
def get_hole(ts, threshold=220*1e-5):
    ttoz = np.array(ds['toz'][ts,:,:])
    #print(threshold)
    #print(ttoz.min(), ttoz.max())
    #print(ttoz.shape)
    below = ttoz < threshold
    hole_area = np.array(grid_nc['m2'])[below]
    m2 = hole_area.sum()
    km2 = 1e-6 * m2
    Mkm2 = 1e-6 * km2
    #print(hole_area.shape, m2, km2, Mkm2)
    return Mkm2
print(get_hole(ts), "million km²")


### Now do this for each timestep, storing the max size per year

In [ ]:
# Takes about 30 min to run, could optimise but only needs to be run once so...

In [ ]:
ds['time']
dates = np.array(ds['time'])
years = dates.astype('datetime64[Y]').astype(int) + 1970
months = dates.astype('datetime64[M]').astype(int) % 12 + 1
days = (dates.astype('datetime64[D]') - dates.astype('datetime64[M]')).astype(int) + 1

import json
import os
cfn = "year_max.json"
afn = "all.json"
alldata = []
if not os.path.exists(cfn):
    year_max = {}
    month_max = {}
    print('Calculating...')
    for i in range(len(dates)):
        #print(years[i], months[i], days[i])
        Mkm2 = get_hole(i)
        #print(hole_area.shape, m2, km2, Mkm2)
        year = str(years[i])
        if not year in year_max or year_max[year] < Mkm2:
            year_max[year] = Mkm2
        alldata.append(Mkm2)
    print('Saving...')
    #print(year_max)
    with open(cfn, "w") as outfile:
        json.dump(year_max, outfile)
    with open(afn, "w") as outfile:
        json.dump(alldata, outfile)        
else:
    with open(cfn, "r") as infile:
        year_max = json.load(infile)
    with open(afn, "r") as infile:
        alldata = json.load(infile)


In [ ]:
import matplotlib.pyplot as plt
x = np.linspace(0, len(alldata)-1, num=len(alldata), endpoint=True)
#print(len(xnew))
#print(len(alldata))
plt.scatter(x, alldata);

In [ ]:
#Get the axis data

#Years, sorted and duplicates removed
x = np.array(sorted(list(set(years))))
#Max hole area per year
y = np.array(list(year_max.values()))

#print(x, y)
#plt.plot(x,y)
#plt.show()

#create scatterplot
plt.bar(x, y)
#plt.scatter(x, y)

https://www.datatechnotes.com/2021/11/scattered-data-spline-fitting-example.html

In [ ]:
from scipy import interpolate
import matplotlib.pyplot as plt
import numpy as np 

 Spline curve fitting
 
    To construct a smoother spline fit, we need to specify the number of knots for the target data. Knots are joints of polynomial segments.
    Based on knots number, we'll determine the new x data vector by using the 'quantile' function. 

In [ ]:
knot_numbers = 2 #Higher numbers fit the data closer, but look bad at the ends
x_new = np.linspace(0, 1, knot_numbers+2)[1:-1]
q_knots = np.quantile(x, x_new) 

    Next, we'll find out the required coefficient values by using 'splrep'. The 'splrep' function returns t, c, k tuple containing the vector of knots, the B-spline coefficients, and the degree of the spline.

    After taking the values, we'll use BSpline class to construct spline fit on x vector data.

In [ ]:
t,c,k = interpolate.splrep(x, y, t=q_knots, s=1)
yfit = interpolate.BSpline(t,c,k)(x) 

    Finally, we can visualize the constructed spline curve on a graph.

In [ ]:
# Enable interactive plot
!pip install ipympl
#%matplotlib notebook
%matplotlib widget

In [ ]:
#print(plt.style.available)
plt.style.use("dark_background")

In [ ]:
from PIL import Image
#size = 720*16//9, 720
size = 640, 480
frame = np.zeros(dtype='uint8', shape=(size[1], size[0], 3))

In [ ]:
# create a figure with axes
fig, ax = plt.subplots()
canvas = fig.canvas

#plt.figure(figsize=(12, 6))
#plt.title("Ozone hole maximum size")
plt.plot(x, y, '.', c="grey", label="original")
plt.plot(x, yfit, '-', c="cyan", label="spline fit")
#plt.legend(loc='best', fancybox=True, shadow=True)
#plt.grid()
plt.xlabel('Year')
plt.ylabel('Area (million km²)')
#plt.show()

# Hide the right and top spines
ax.spines[['right', 'top']].set_visible(False)
#Remove axis margins
ax.margins(x=0)
ax.margins(y=0)
#Hide intermediate ticks
xlabels = ax.xaxis.get_ticklabels()
for xl in xlabels[1:-1]:
    xl.set_visible(False)
ylabels = ax.yaxis.get_ticklabels()
for yl in ylabels[0:-2]:
    yl.set_visible(False)
plt.ylim([0, 20])

#ax = plt.gca()
# create a point in the axes
point, = ax.plot(years[0],y[0], marker="o", color='yellow', markersize=10)
point0, = ax.plot(years[0],y[0], marker="o", color='white', markersize=5)
#bar, = ax.bar(1960,y[0], color='darkgrey')

# Updating function, to be repeatedly called by the animation
from matplotlib.animation import FuncAnimation
X = None
def update(ts):
    global X
    # set point's coordinates
    point0.set_data([1960+ts],[y[ts]])
    point.set_data([1960+ts],[yfit[ts]])
    return point,point0

# create animation with 10ms interval, which is repeated,
# provide the timestep index as a parameter
ani = FuncAnimation(fig, update, interval=10, blit=True, repeat=True,
                    frames=range(0,len(y)))

plt.show();